In [1]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

from vit_pytorch.efficient import ViT
from vit_pytorch.cvt import CvT


In [2]:
PATH="./trained_models/pretrained_net"+"47"+".pt"
# Training settings
batch_size = 1
device = 'cuda'

In [3]:

from vit_pytorch.cross_vit import CrossViT

model = CrossViT(
    image_size = 224,
    num_classes = 4,
    depth = 4,               # number of multi-scale encoding blocks
    sm_dim = 192,            # high res dimension
    sm_patch_size = 16,      # high res patch size (should be smaller than lg_patch_size)
    sm_enc_depth = 2,        # high res depth
    sm_enc_heads = 8,        # high res heads
    sm_enc_mlp_dim = 2048,   # high res feedforward dimension
    lg_dim = 384,            # low res dimension
    lg_patch_size = 32,      # low res patch size
    lg_enc_depth = 3,        # low res depth
    lg_enc_heads = 8,        # low res heads
    lg_enc_mlp_dim = 2048,   # low res feedforward dimensions
    cross_attn_depth = 2,    # cross attention rounds
    cross_attn_heads = 8,    # cross attention heads
    dropout = 0.1,
    emb_dropout = 0.1
).to(device)
model.load_state_dict(torch.load(PATH))
model.eval()

CrossViT(
  (sm_image_embedder): ImageEmbedder(
    (to_patch_embedding): Sequential(
      (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=16, p2=16)
      (1): Linear(in_features=768, out_features=192, bias=True)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (lg_image_embedder): ImageEmbedder(
    (to_patch_embedding): Sequential(
      (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=32, p2=32)
      (1): Linear(in_features=3072, out_features=384, bias=True)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (multi_scale_encoder): MultiScaleEncoder(
    (layers): ModuleList(
      (0): ModuleList(
        (0): Transformer(
          (layers): ModuleList(
            (0): ModuleList(
              (0): PreNorm(
                (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
                (fn): Attention(
                  (attend): Softmax(dim=-1)
                  (to_q): Linear(in_features=192, out_features=512, bias=False)
  

In [4]:
test_dir = 'agumented'
test_list = glob.glob(os.path.join(test_dir, '*.jpg'))
print(f"Test Data: {len(test_list)}")

Test Data: 1118


In [5]:
labels = [1 for path in test_list]
#print(labels)

In [6]:
test_transforms = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]
)

In [7]:
class CropsDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        label = 1
        
        return img_transformed, label, img_path

In [8]:
test_data = CropsDataset(test_list, transform=test_transforms)
test_loader = DataLoader(dataset = test_data, batch_size=batch_size, shuffle=False)

In [9]:
def append_new_line(file_name, text_to_append):
    """Append given text as a new line at the end of file"""
    # Open the file in append & read mode ('a+')
    with open(file_name, "a+") as file_object:
        # Move read cursor to the start of file.
        file_object.seek(0)
        # If file is not empty then append '\n'
        data = file_object.read(100)
        if len(data) > 0:
            file_object.write("\n")
        # Append text at the end of file
        file_object.write(text_to_append)

In [10]:
print("Predictions: ")
file_object = open('sample7.txt', 'a')
for data, label,img_path in tqdm(test_loader):
        data = data.to(device)
        label = label.cpu()
        output = model(data).max(1).indices.cpu().numpy()[0]
        img_path=str(img_path)
        img_path=img_path[:-3];
        img_path=img_path[2:];
        line=str(img_path)+","+str(output)
        append_new_line('sample7.txt', line)
        print("Predicticted: ",line)
file_object.close()        

Predictions: 


  0%|          | 0/1118 [00:00<?, ?it/s]

Predicticted:  agumented\\1305121398-0001-R_S.jpg,3
Predicticted:  agumented\\1305121398-0001-R_S2.jpg,3
Predicticted:  agumented\\1305121398-0001-R_S3.jpg,3
Predicticted:  agumented\\1305121398-0003-R.jpg,3
Predicticted:  agumented\\1305121398-0004-R.jpg,1
Predicticted:  agumented\\1305121398-0004-R2.jpg,3
Predicticted:  agumented\\1305121398-0005-T.jpg,1
Predicticted:  agumented\\1305121398-0006-R.jpg,1
Predicticted:  agumented\\1305121398-0007-R.jpg,1
Predicticted:  agumented\\1305121398-0008-R.jpg,1
Predicticted:  agumented\\1305121398-0010-S.jpg,2
Predicticted:  agumented\\1305121398-0011-S.jpg,2
Predicticted:  agumented\\1305121398-0012-S.jpg,2
Predicticted:  agumented\\1305121398-0013-S.jpg,2
Predicticted:  agumented\\1305121398-0014-S.jpg,2
Predicticted:  agumented\\1305121398-0015-S.jpg,2
Predicticted:  agumented\\1305121398-0016-S.jpg,2
Predicticted:  agumented\\1305121398-0017-R.jpg,1
Predicticted:  agumented\\1305121398-0018-S.jpg,2
Predicticted:  agumented\\1305121398-0019

Predicticted:  agumented\\1307210661-0006-R17.jpg,1
Predicticted:  agumented\\1307210661-0006-R18.jpg,3
Predicticted:  agumented\\1307210661-0006-R19.jpg,1
Predicticted:  agumented\\1307210661-0006-R2.jpg,3
Predicticted:  agumented\\1307210661-0006-R20.jpg,1
Predicticted:  agumented\\1307210661-0006-R21.jpg,1
Predicticted:  agumented\\1307210661-0006-R22.jpg,1
Predicticted:  agumented\\1307210661-0006-R23.jpg,1
Predicticted:  agumented\\1307210661-0006-R24.jpg,0
Predicticted:  agumented\\1307210661-0006-R25.jpg,1
Predicticted:  agumented\\1307210661-0006-R3.jpg,1
Predicticted:  agumented\\1307210661-0006-R4.jpg,1
Predicticted:  agumented\\1307210661-0006-R5.jpg,1
Predicticted:  agumented\\1307210661-0006-R6.jpg,3
Predicticted:  agumented\\1307210661-0006-R7.jpg,3
Predicticted:  agumented\\1307210661-0006-R8.jpg,1
Predicticted:  agumented\\1307210661-0006-R9.jpg,3
Predicticted:  agumented\\1307210661-0007-R.jpg,1
Predicticted:  agumented\\1307210661-0007-R10.jpg,1
Predicticted:  agument

Predicticted:  agumented\\1405022890-0003-R5.jpg,1
Predicticted:  agumented\\1405022890-0003-R6.jpg,1
Predicticted:  agumented\\1405022890-0003-R7.jpg,1
Predicticted:  agumented\\1405022890-0003-R8.jpg,1
Predicticted:  agumented\\1405022890-0003-R9.jpg,3
Predicticted:  agumented\\1405022890-0004-R.jpg,1
Predicticted:  agumented\\1405022890-0004-R10.jpg,1
Predicticted:  agumented\\1405022890-0004-R11.jpg,1
Predicticted:  agumented\\1405022890-0004-R12.jpg,1
Predicticted:  agumented\\1405022890-0004-R13.jpg,3
Predicticted:  agumented\\1405022890-0004-R14.jpg,3
Predicticted:  agumented\\1405022890-0004-R15.jpg,1
Predicticted:  agumented\\1405022890-0004-R16.jpg,0
Predicticted:  agumented\\1405022890-0004-R17.jpg,1
Predicticted:  agumented\\1405022890-0004-R18.jpg,1
Predicticted:  agumented\\1405022890-0004-R19.jpg,1
Predicticted:  agumented\\1405022890-0004-R2.jpg,1
Predicticted:  agumented\\1405022890-0004-R3.jpg,0
Predicticted:  agumented\\1405022890-0004-R4.jpg,1
Predicticted:  agument

Predicticted:  agumented\\1701151546-0001-R_T_S3.jpg,2
Predicticted:  agumented\\1701151546-0001-R_T_S4.jpg,3
Predicticted:  agumented\\1701151546-0001-R_T_S5.jpg,3
Predicticted:  agumented\\1701151546-0001-R_T_S6.jpg,1
Predicticted:  agumented\\1701151546-0002-R.jpg,3
Predicticted:  agumented\\1701151546-0002-R10.jpg,1
Predicticted:  agumented\\1701151546-0002-R11.jpg,1
Predicticted:  agumented\\1701151546-0002-R12.jpg,1
Predicticted:  agumented\\1701151546-0002-R13.jpg,1
Predicticted:  agumented\\1701151546-0002-R14.jpg,1
Predicticted:  agumented\\1701151546-0002-R15.jpg,1
Predicticted:  agumented\\1701151546-0002-R16.jpg,1
Predicticted:  agumented\\1701151546-0002-R2.jpg,3
Predicticted:  agumented\\1701151546-0002-R3.jpg,3
Predicticted:  agumented\\1701151546-0002-R4.jpg,3
Predicticted:  agumented\\1701151546-0002-R5.jpg,3
Predicticted:  agumented\\1701151546-0002-R6.jpg,1
Predicticted:  agumented\\1701151546-0002-R7.jpg,3
Predicticted:  agumented\\1701151546-0002-R8.jpg,3
Predictic

Predicticted:  agumented\\1701151546-0011-R_T12.jpg,1
Predicticted:  agumented\\1701151546-0011-R_T13.jpg,1
Predicticted:  agumented\\1701151546-0011-R_T14.jpg,1
Predicticted:  agumented\\1701151546-0011-R_T15.jpg,1
Predicticted:  agumented\\1701151546-0011-R_T16.jpg,1
Predicticted:  agumented\\1701151546-0011-R_T17.jpg,1
Predicticted:  agumented\\1701151546-0011-R_T2.jpg,2
Predicticted:  agumented\\1701151546-0011-R_T3.jpg,3
Predicticted:  agumented\\1701151546-0011-R_T4.jpg,1
Predicticted:  agumented\\1701151546-0011-R_T5.jpg,1
Predicticted:  agumented\\1701151546-0011-R_T6.jpg,2
Predicticted:  agumented\\1701151546-0011-R_T7.jpg,1
Predicticted:  agumented\\1701151546-0011-R_T8.jpg,3
Predicticted:  agumented\\1701151546-0011-R_T9.jpg,3
Predicticted:  agumented\\1701151546-0012-R.jpg,1
Predicticted:  agumented\\1701151546-0012-R10.jpg,1
Predicticted:  agumented\\1701151546-0012-R11.jpg,1
Predicticted:  agumented\\1701151546-0012-R12.jpg,1
Predicticted:  agumented\\1701151546-0012-R13.

Predicticted:  agumented\\1704282807-0003-R6.jpg,1
Predicticted:  agumented\\1704282807-0003-R7.jpg,1
Predicticted:  agumented\\1704282807-0003-R8.jpg,1
Predicticted:  agumented\\1704282807-0003-R9.jpg,1
Predicticted:  agumented\\1704282807-0004-R_T.jpg,1
Predicticted:  agumented\\1704282807-0004-R_T10.jpg,1
Predicticted:  agumented\\1704282807-0004-R_T11.jpg,3
Predicticted:  agumented\\1704282807-0004-R_T12.jpg,1
Predicticted:  agumented\\1704282807-0004-R_T13.jpg,1
Predicticted:  agumented\\1704282807-0004-R_T14.jpg,1
Predicticted:  agumented\\1704282807-0004-R_T2.jpg,1
Predicticted:  agumented\\1704282807-0004-R_T3.jpg,1
Predicticted:  agumented\\1704282807-0004-R_T4.jpg,1
Predicticted:  agumented\\1704282807-0004-R_T5.jpg,1
Predicticted:  agumented\\1704282807-0004-R_T6.jpg,2
Predicticted:  agumented\\1704282807-0004-R_T7.jpg,1
Predicticted:  agumented\\1704282807-0004-R_T8.jpg,1
Predicticted:  agumented\\1704282807-0004-R_T9.jpg,1
Predicticted:  agumented\\1704282807-0005-R.jpg,1


Predicticted:  agumented\\1704282807-0013-R_T7.jpg,1
Predicticted:  agumented\\1704282807-0013-R_T8.jpg,1
Predicticted:  agumented\\1704282807-0013-R_T9.jpg,1
Predicticted:  agumented\\1704282807-0014-R.jpg,1
Predicticted:  agumented\\1704282807-0014-R10.jpg,1
Predicticted:  agumented\\1704282807-0014-R11.jpg,1
Predicticted:  agumented\\1704282807-0014-R12.jpg,1
Predicticted:  agumented\\1704282807-0014-R13.jpg,1
Predicticted:  agumented\\1704282807-0014-R14.jpg,1
Predicticted:  agumented\\1704282807-0014-R2.jpg,1
Predicticted:  agumented\\1704282807-0014-R3.jpg,1
Predicticted:  agumented\\1704282807-0014-R4.jpg,1
Predicticted:  agumented\\1704282807-0014-R5.jpg,1
Predicticted:  agumented\\1704282807-0014-R6.jpg,3
Predicticted:  agumented\\1704282807-0014-R7.jpg,1
Predicticted:  agumented\\1704282807-0014-R8.jpg,1
Predicticted:  agumented\\1704282807-0014-R9.jpg,2
Predicticted:  agumented\\1704282807-0015-R.jpg,1
Predicticted:  agumented\\1704282807-0015-R10.jpg,3
Predicticted:  agumen

In [11]:
#file_object.close()  